In [7]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
from lxml import html
  

In [9]:
''' BEAUTIFUL SOUP '''

def getHTMLdocument(url):
      
        response = requests.get(url)      
        return response.text

rownum = 0
book_table = pd.DataFrame(columns=["Sr. No.", "Name", "Link", "Cost", "Availability"])

start_time_bs = time.process_time()
start_time_total_bs = time.time()

url_to_scrape = "http://books.toscrape.com/catalogue/page-" + str(1) + ".html"  
html_document = getHTMLdocument(url_to_scrape)  
soup = BeautifulSoup(html_document, 'html.parser')


for row in soup.find_all('li', attrs={'class': re.compile("col-xs-6")}):

        rownum += 1
        name = row.find('h3').find('a').get('title')
        link = "http://books.toscrape.com/catalogue/" + row.find('h3').find('a').get('href')
        cost = row.find('p', attrs = {'class': re.compile("price_color")}).text.strip()
        cost = cost[2:]
        availability = row.find('p', attrs = {'class': re.compile("instock availability")}).text.strip()        
        book_table.loc[len(book_table.index)] = [rownum, name, link, cost, availability ] 

stop_time_bs = time.process_time()
stop_time_total_bs = time.time()

print(f"Process Time Taken - beautifulsoup: {stop_time_bs - start_time_bs} sec")
print(f"Total Time Taken - beautifulsoup: {stop_time_total_bs - start_time_total_bs} sec")

book_table.to_excel('py_request_bs.xlsx',index=False)

Process Time Taken - beautifulsoup: 0.171875 sec
Total Time Taken - beautifulsoup: 0.6300647258758545 sec


In [14]:
''' LXML '''

def getHTMLdocument(url):      
        response = requests.get(url)      
        return response.content

rownum = 0
book_table_lxml = pd.DataFrame(columns=["Sr. No.", "Name", "Link", "Cost", "Availability"])

start_time_lxml = time.process_time()
start_time_total_lxml = time.time()

url = "http://books.toscrape.com/catalogue/page-" + str(1) + ".html"
byte_data = getHTMLdocument(url)
source_code = html.fromstring(byte_data)
    
for i in range(1,21):

        rownum += 1
        name_path = '//*[@id="default"]/div/div/div/div/section/div[2]/ol/li[' + str(i) + ']/article/h3/a/@title' 
        book_link_path = '//*[@id="default"]/div/div/div/div/section/div[2]/ol/li[' + str(i) + ']/article/h3/a/@href'
        cost_path = '//*[@id="default"]/div/div/div/div/section/div[2]/ol/li[' + str(i) + ']/article/div[2]/p[1]' 
        stock_path = '//*[@id="default"]/div/div/div/div/section/div[2]/ol/li[' + str(i) + ']/article/div[2]/p[2]' 

        name = source_code.xpath(name_path)[0]
        book_link = "http://books.toscrape.com/catalogue/" + source_code.xpath(book_link_path)[0]
        cost = source_code.xpath(cost_path)[0].text_content()
        stock = source_code.xpath(stock_path)[0].text_content().strip()

        book_table_lxml.loc[len(book_table_lxml.index)] = [rownum, name, book_link, cost, stock ]

stop_time_lxml = time.process_time()
stop_time_total_lxml = time.time()


print(f"Process Time Taken - lxml: {stop_time_lxml - start_time_lxml}")
print(f"Total Time Taken - lxml: {stop_time_total_lxml - start_time_total_lxml} sec")

book_table_lxml.to_excel('py_request_lxml.xlsx', index = False)           

Process Time Taken - lxml: 0.125
Total Time Taken - lxml: 0.676447868347168 sec
